In [1]:
!pip install geopandas
!pip install folium
!pip install geopy
!pip install osmnx==1.9.1
!pip install --upgrade osmnx matplotlib
!pip -m pip install redis

     |████████████████████████████████| 1.1 MB 24.0 MB/s eta 0:00:01
     |████████████████████████████████| 8.3 MB 35.9 MB/s eta 0:00:01
     |████████████████████████████████| 15.7 MB 47.8 MB/s eta 0:00:01
     |████████████████████████████████| 13.1 MB 51.0 MB/s eta 0:00:01
     |████████████████████████████████| 2.5 MB 44.3 MB/s eta 0:00:01
     |████████████████████████████████| 18.2 MB 49.4 MB/s eta 0:00:01
     |████████████████████████████████| 345 kB 58.9 MB/s eta 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.20.3
    Uninstalling numpy-1.20.3:
      Successfully uninstalled numpy-1.20.3
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.4
    Uninstalling pandas-1.3.4:
      Successfully uninstalled pandas-1.3.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.3.0 requires daal==2021

  Attempting uninstall: requests
    Found existing installation: requests 2.26.0
    Uninstalling requests-2.26.0:
      Successfully uninstalled requests-2.26.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.4 requires pathlib, which is not installed.
anaconda-project 0.10.1 requires ruamel-yaml, which is not installed.
cookiecutter 1.7.2 requires Jinja2<3.0.0, but you have jinja2 3.1.3 which is incompatible.
cookiecutter 1.7.2 requires MarkupSafe<2.0.0, but you have markupsafe 2.1.5 which is incompatible.
     |████████████████████████████████| 107 kB 14.3 MB/s eta 0:00:01
     |████████████████████████████████| 11.6 MB 57.9 MB/s eta 0:00:01
     |████████████████████████████████| 304 kB 58.5 MB/s eta 0:00:01
  Attempting uninstall: osmnx
    Found existing installation: osmnx 1.9.1
    Uninstalling osmnx-1.9.1:
      Successfully uninsta

Everything above was testing if redis works in the environment

In [2]:
import redis
import json
import pandas as pd
import geopandas as gpd
import math
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
from geopy.geocoders import Nominatim
import random
import time
import osmnx as ox
from shapely.geometry import Point, LineString
import pickle as pk
import os
from IPython.display import clear_output

/opt/conda/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/opt/conda/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (
/opt/conda/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
#Will change the db number later when I clean up the notebook
#change host = '' value to redis container ID when running docker in the ananconda_postgres_redis folder
client = redis.StrictRedis(host='9b1ed92813b7', port=6379, db=1)

with open('county_routes.json') as data_file:
    test_data1 = json.load(data_file)
    test_data1 = json.dumps(test_data1)
client.set('county_json', test_data1)

with open('store_info.json') as data_file:
    test_data2 = json.load(data_file)
    test_data2 = json.dumps(test_data2)
client.set('store_json', test_data2)

True

In [4]:
county_data = client.get('county_json')

In [5]:
store_data = client.get('store_json')

In [6]:
county_routes = json.loads(county_data)
store_info = json.loads(store_data)

This will now get the central locations and assign the counties to these coordinates

In [7]:
counties = ['San Francisco', 'Santa Clara', 'Contra Costa', 'Alameda', 'San Mateo']
coordinates = [[37.759344, -122.446452], [37.258336, -121.734320], [37.955321, -121.879418], 
               [37.674112, -121.971002], [37.466796, -122.391435]]

In [8]:
folium_maps = {}

for i, county in enumerate(counties):
    folium_maps[county] = folium.Map(location=coordinates[i], tiles='openstreetmap', prefer_canvas=True)

This code will take a while to run as it needs to generate the maps for each of the counties

In [9]:
maps = {}

for county in counties:
    maps[county] = ox.graph_from_place(f"{county}, California, USA", network_type="drive")
    
for county, graph in maps.items():
    file_name = f"{county}_graph.pickle"
    file_path = os.path.join('.', file_name)
    with open(file_path, 'wb') as handle:
        pk.dump(graph, handle, protocol=pk.HIGHEST_PROTOCOL)

In [23]:
ALL_routes = county_routes[county]['route']

In [24]:
counties[3]

'Alameda'

In [25]:
Alameda = folium.Map(location=coordinates[3], tiles='openstreetmap', prefer_canvas=True)

This is the main code to run and get to see how the driver moves along the road to their next destination.

DISCLAIMER: THERE IS A BUG THAT WHEN YOU RUN THROUGH THE NOTEBOOK THE FIRST TIME.
THIS CODE WILL NOT RUN YOU WILL HAVE TO RE-RUN THE ABOVE 3 CODE LINES AGAIN.
BEFORE IT WORKS AGAIN.
CURRENTLY WORKING ON FIXING IT

In [32]:
from random import randint
G = maps['Alameda']
county = 'Alameda'
names = county_routes[county]['nodes']
map_ = Alameda


switch = True
for j, route in enumerate(ALL_routes):
    initialize = []
    breaker = True
    if switch == True:
        initialize.append(route[0])
        initialize.append(route[1])
        store_cords = county_routes[county]['coordinates'][j]
        lon_str, lat_str = store_cords.strip('()').split(', ')
        map_ = folium.Map(location=coordinates[3], tiles='openstreetmap', prefer_canvas=True)
        folium.Marker([float(lon_str), float(lat_str)], popup=names[j], icon=folium.Icon(color='red')).add_to(map_)
        print(f"Viewing Route for {county} County | Currently enroute to {names[j+1]} from {names[j]}")
        map_= ox.plot_route_folium(G, initialize, route_map=map_, weight=10)
        map_.fit_bounds(map_.get_bounds())
        display(map_)
        switch = False
    s = 2
    while s < len(route):
        print(f"Viewing Route for {county} County | Currently enroute to {names[j]} from {names[j+1]}")
        if s == (len(route) - 1):
            map_= ox.plot_route_folium(G, route, route_map=map_, weight=10)
            map_.fit_bounds(map_.get_bounds())
            display(map_)
            del initialize[:]
            switch = True
            break
        elif breaker == True:
            popup_text = f"<b>{names[j]}</b><br>Unloading...<br><br><b>Delivery Status:</b><br>"
            items_delivered = []
            for item, quantity in store_info[names[j]]['items_requested'].items():
#                     print(len(store_info[names[j]]['items_requested'].items()))
#                     print(item)
#                     print(store_info[names[j]]['items_requested'])
#                     print(quantity)
#                     if quantity > 0:
#                         if quantity == 1:
#                             popup_text += f"{item}: Delivered<br>"
#                             items_delivered.append(item)
#                         else:
#                             popup_text += f"{item}: {quantity - 1} remaining<br>"
#                         store_info[names[j]]['items_requested'][item] -= 1
                clear_output(wait=True)
                popup_text += f"{item}: Delivered<br>"
                items_delivered.append(item)
                store_info[names[j]]['items_requested'][item] -= quantity
                time.sleep(randint(7,15))

                marker = folium.Marker([float(lon_str), float(lat_str)], popup=popup_text, icon=folium.Icon(color='red'))
                marker.add_to(map_)
                map_.fit_bounds(map_.get_bounds())
                display(map_)

                if (len(items_delivered) == len(store_info[names[j]]['items_requested'].items())):
                    clear_output(wait=True)
                    popup_text = f"<b>{names[j]}</b><br>Completed"
                    breaker = False
                    marker = folium.Marker([float(lon_str), float(lat_str)], popup=popup_text, icon=folium.Icon(color='red'))
                    marker.add_to(map_)
                    map_.fit_bounds(map_.get_bounds())
                    display(map_)

            for item_delivered in items_delivered:
                del store_info[names[j]]['items_requested'][item_delivered]

            client.set('store_json', json.dumps(store_info))

        else:
#                 print('Before: ', initialize)
            clear_output(wait=True)
            map_= ox.plot_route_folium(G, initialize, route_map=map_, weight=10)
            map_.fit_bounds(map_.get_bounds())
            display(ox.plot_route_folium(G, initialize, route_map=map_, weight=10))
#                 print('Route: ', route[s])
#                 print(type(route))
            initialize.append(route[s])
#                 print('After: ', initialize)

            s+=1
        time.sleep(3)



KeyboardInterrupt: 